In [6]:
import pennylane as qml
from pennylane import numpy as np
from pennylane.qchem import molecular_hamiltonian
from pennylane import commutator
from scipy.linalg import expm
import matplotlib.pyplot as plt

# ========== 1. 分子设置 ==========
import pennylane as qml
from pennylane import numpy as np
from pennylane.qchem import molecule
from pennylane import commutator
import matplotlib.pyplot as plt

import pennylane as qml
from pennylane import numpy as np
from pennylane.qchem import molecule
from pennylane import commutator
import matplotlib.pyplot as plt

# 1. 设置分子结构
from pennylane.qchem import molecular_hamiltonian

from pyscf import gto, scf
from openfermionpyscf import run_pyscf
from openfermion.transforms import jordan_wigner
from openfermion.linalg import get_sparse_operator
import scipy

from openfermion import MolecularData
from openfermionpyscf import run_pyscf
from openfermion.transforms import jordan_wigner
from openfermion.linalg import get_sparse_operator
import pylab
import concurrent.futures
import multiprocessing as mp
import numpy as np
from scipy.linalg import expm

from qiskit.circuit.library import UnitaryGate
from qiskit.circuit.classical import expr
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit.visualization import plot_histogram
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit_aer.primitives import Sampler

from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.mappers import ParityMapper
from qiskit_nature.second_q.algorithms import GroundStateEigensolver
from qiskit_nature.second_q.circuit.library import HartreeFock

from qiskit_algorithms import NumPyMinimumEigensolver
from qiskit_algorithms import IterativePhaseEstimation

from qiskit_ibm_runtime import QiskitRuntimeService
import time
import pylab
import concurrent.futures
import multiprocessing as mp
import numpy as np
from scipy.linalg import expm

from qiskit.circuit.library import UnitaryGate
from qiskit.circuit.classical import expr
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit.visualization import plot_histogram
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit_aer.primitives import Sampler

from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.mappers import ParityMapper
from qiskit_nature.second_q.algorithms import GroundStateEigensolver
from qiskit_nature.second_q.circuit.library import HartreeFock

from qiskit_algorithms import NumPyMinimumEigensolver
from qiskit_algorithms import IterativePhaseEstimation

from qiskit_ibm_runtime import QiskitRuntimeService




In [36]:
from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.drivers import PySCFDriver

driver = PySCFDriver(
    atom="H 0 0 0; H 0 0 0.74",
    basis="sto3g",
    charge=0,
    spin=0,
    unit=DistanceUnit.ANGSTROM,
)

In [40]:
molecule = driver.run()
mapper = ParityMapper()
qubit_op = mapper.map(molecule.hamiltonian.second_q_op())
print(qubit_op)
cost_h=qubit_op

SparsePauliOp(['IIII', 'IIIZ', 'IIZZ', 'IZZI', 'ZZII', 'IIZI', 'IZZZ', 'ZZIZ', 'ZXIX', 'IXZX', 'ZXZX', 'IXIX', 'IZIZ', 'ZZZZ', 'ZIZI'],
              coeffs=[-0.81217061+0.j,  0.17141283+0.j, -0.22343154+0.j,  0.17141283+0.j,
 -0.22343154+0.j,  0.12062523+0.j,  0.16868898+0.j,  0.16592785+0.j,
  0.04530262+0.j, -0.04530262+0.j, -0.04530262+0.j,  0.04530262+0.j,
  0.16592785+0.j,  0.17441288+0.j,  0.12062523+0.j])


In [42]:
from qiskit.quantum_info import SparsePauliOp

driver_h = SparsePauliOp.from_list([
    ("XIII", -1.0),
    ("IXII", -1.0),
    ("IIXI", -1.0),
    ("IIIX", -1.0),
])
print(driver_h)

SparsePauliOp(['XIII', 'IXII', 'IIXI', 'IIIX'],
              coeffs=[-1.+0.j, -1.+0.j, -1.+0.j, -1.+0.j])


In [68]:
from qiskit.quantum_info import SparsePauliOp

def build_commutator(op_a: SparsePauliOp, op_b: SparsePauliOp) -> SparsePauliOp:
    """
    Compute the commutator [A, B] = AB - BA for two SparsePauliOp Hamiltonians.

    Args:
        op_a (SparsePauliOp): First operator A.
        op_b (SparsePauliOp): Second operator B.

    Returns:
        SparsePauliOp: The commutator [A, B].
    """
    ab = op_a @ op_b
    ba = op_b @ op_a
    comm_pre= ab - ba
    comm = comm_pre*-1j
    return comm.simplify()


In [67]:
comm_h=build_commutator(driver_h, cost_h)
print(comm_h)



SparsePauliOp(['YZII', 'YZIZ', 'YXIX', 'YXZX', 'YZZZ', 'YIZI', 'IYZI', 'ZYII', 'IYZZ', 'ZYIZ', 'IYIZ', 'ZYZZ', 'IIYZ', 'IZYI', 'IIYI', 'IZYZ', 'IXYX', 'ZXYX', 'ZZYZ', 'ZIYI', 'IIIY', 'IIZY', 'IZZY', 'ZZIY', 'IZIY', 'ZZZY'],
              coeffs=[ 0.44686307+0.j, -0.3318557 +0.j, -0.09060523+0.j,  0.09060523+0.j,
 -0.34882575+0.j, -0.24125047+0.j, -0.34282565+0.j,  0.44686307+0.j,
 -0.33737796+0.j, -0.3318557 +0.j, -0.3318557 +0.j, -0.34882575+0.j,
  0.44686307+0.j, -0.34282565+0.j, -0.24125047+0.j, -0.33737796+0.j,
  0.09060523+0.j,  0.09060523+0.j, -0.34882575+0.j, -0.24125047+0.j,
 -0.34282565+0.j,  0.44686307+0.j, -0.33737796+0.j, -0.3318557 +0.j,
 -0.3318557 +0.j, -0.34882575+0.j])


In [77]:
from qiskit import QuantumCircuit
from qiskit.quantum_info import SparsePauliOp
import numpy as np
import matplotlib.pyplot as plt
from qiskit.quantum_info import Statevector
# 构造 FALQON 电路的一层
def falqon_layer_qiskit(beta_k, cost_h, driver_h, delta_t, num_qubits):
    qc = QuantumCircuit(num_qubits)
    qc.h(range(num_qubits))  # 初始化为均匀叠加态

    # 模拟 cost Hamiltonian 的演化 e^{-i H_c ∆t}
    for pauli, coeff in zip(cost_h.paulis, cost_h.coeffs):
        if not np.isclose(coeff, 0.0):
            indices = [i for i, p in enumerate(pauli.to_label()) if p != 'I']
            angle = 2 * coeff.real * delta_t
            for idx in indices:
                qc.rz(angle, idx)  # 简化为 RZ；复杂项需多门组合
                # 实际可构建 exp(-i a P t) 形式，用 PauliRot 构造器逻辑

    # 模拟 driver Hamiltonian 的演化 e^{-i β_k H_d ∆t}
    for pauli, coeff in zip(driver_h.paulis, driver_h.coeffs):
        if not np.isclose(coeff, 0.0):
            indices = [i for i, p in enumerate(pauli.to_label()) if p != 'I']
            angle = 2 * coeff.real * beta_k * delta_t
            for idx in indices:
                qc.rx(angle, idx)

    return qc

# 主循环：仅结构展示，测量结果为随机占位
def falqon_qiskit(n, beta_1, delta_t, cost_h, driver_h, comm_h, num_qubits):
    betas = [beta_1]
    energies = []

    for i in range(n):
        beta_k = betas[-1]
        qc = falqon_layer_qiskit(beta_k, cost_h, driver_h, delta_t, num_qubits)
        # TODO：这里你可以插入真实测量流程，比如 Sampler + SparsePauliOp.expval
        state = Statevector.from_instruction(qc)
        energy = np.real(state.expectation_value(cost_h))
        betas.append(-1 * dummy_energy)  # feedback 更新 beta_k

        print(f"Step {i+1}: Energy = {dummy_energy:.6f}")

    return betas, energies

# 假设你已经准备好了以下变量
# cost_h = SparsePauliOp(...)
# driver_h = SparsePauliOp(...)
# comm_h = SparsePauliOp(...)
num_qubits = 4
n = 100
beta_1 = 0.01
delta_t = 0.03

res_beta, res_energies = falqon_qiskit(n, beta_1, delta_t, cost_h, driver_h, comm_h, num_qubits)

# 绘图
plt.plot(range(1, len(res_energies)+1), res_energies)
plt.xlabel("Iteration")
plt.ylabel("Cost Function Value (placeholder)")
plt.title("Qiskit FALQON Energy Convergence")
plt.grid(True)
plt.show()


NameError: name 'dummy_energy' is not defined